# JOINS

Bildiğimiz gibi çok büyük veri setleri üzerinde çalışırken bizim için en maliyetli işlemlerden biri join işlemleridir.Bu kısımda join işlemlerinin spark üzerinde nasıl çalıştırıldığına ve kullanılacağına bakacağız.Öncelikle çok fazla join çeşidi var ama zaten biri diğerinin laciverti,daha çok biz büyük verilerde ki join işemlerinin bir clusterın üzerinde nasıl çalışırılacağına bakacağız.

Öncelikle join bildiğiniz üzere şartlı kartezyen çarpım olarak anlatılabilir.yani iki veri setini biribiri ile uygun olan bir veya daha fazla şarta göre birleştirme diyebiliriz.

In [16]:
person=spark.createDataFrame\
([(0,"Abdullah ",0,[100])\
 ,(1,"Harun ",1,[500,250,100])\
 ,(2,"Omer ",1,[250,100])])\
.toDF("id","name","graduateProgram","spark program")

In [17]:
graduateProgram=spark.createDataFrame\
([(0,"Master","School of Information","UC Berkeley")\
 ,(2,"Master","EECS","UC Berkeley")\
 ,(1,"PhD","EECS","UC Berkeley")])\
 .toDF("id","degree","department","school")

In [18]:
sparkStatus=spark.createDataFrame\
([(500,"Vice President"),(250,"PMC Member"),(100,"Contributor")]).toDF("id","status")

In [19]:
person.createOrReplaceGlobalTempView("person")
graduateProgram.createOrReplaceGlobalTempView("graduateProgram")
sparkStatus.createOrReplaceGlobalTempView("sparkStatus")
person.select("*").show()
graduateProgram.select("*").show()
sparkStatus.select("*").show()

+---+---------+---------------+---------------+
| id|     name|graduateProgram|  spark program|
+---+---------+---------------+---------------+
|  0|Abdullah |              0|          [100]|
|  1|   Harun |              1|[500, 250, 100]|
|  2|    Omer |              1|     [250, 100]|
+---+---------+---------------+---------------+

+---+------+--------------------+-----------+
| id|degree|          department|     school|
+---+------+--------------------+-----------+
|  0|Master|School of Informa...|UC Berkeley|
|  2|Master|                EECS|UC Berkeley|
|  1|   PhD|                EECS|UC Berkeley|
+---+------+--------------------+-----------+

+---+--------------+
| id|        status|
+---+--------------+
|500|Vice President|
|250|    PMC Member|
|100|   Contributor|
+---+--------------+



Şimdi yukarıda join işlemleri için kullancağımız veri setlerini hazırladık ve hepsini birer temp view haline getirip direk   sql sorguları içinde müsait hale getirmiş olduk.
<p> Aşağıda ise görüdğünüz gibi join işlemlerinin basitce nasıl kullanıldığını görmüş olduk.

In [20]:
#INNER JOIN
joinExpression=person["graduateProgram"]==graduateProgram["id"]
person.join(graduateProgram,joinExpression).show()


+---+---------+---------------+---------------+---+------+--------------------+-----------+
| id|     name|graduateProgram|  spark program| id|degree|          department|     school|
+---+---------+---------------+---------------+---+------+--------------------+-----------+
|  0|Abdullah |              0|          [100]|  0|Master|School of Informa...|UC Berkeley|
|  1|   Harun |              1|[500, 250, 100]|  1|   PhD|                EECS|UC Berkeley|
|  2|    Omer |              1|     [250, 100]|  1|   PhD|                EECS|UC Berkeley|
+---+---------+---------------+---------------+---+------+--------------------+-----------+



In [21]:
#OUTER 
jointype="outer"
person.join(graduateProgram,joinExpression,jointype).show()

+----+---------+---------------+---------------+---+------+--------------------+-----------+
|  id|     name|graduateProgram|  spark program| id|degree|          department|     school|
+----+---------+---------------+---------------+---+------+--------------------+-----------+
|   0|Abdullah |              0|          [100]|  0|Master|School of Informa...|UC Berkeley|
|   1|   Harun |              1|[500, 250, 100]|  1|   PhD|                EECS|UC Berkeley|
|   2|    Omer |              1|     [250, 100]|  1|   PhD|                EECS|UC Berkeley|
|null|     null|           null|           null|  2|Master|                EECS|UC Berkeley|
+----+---------+---------------+---------------+---+------+--------------------+-----------+



In [22]:
#LEFT-RIGHT OUTER JOIN
jointype="left_outer"
graduateProgram.join(person,joinExpression,jointype).show()
jointype="right_outer"
person.join(graduateProgram,joinExpression,jointype).show()

+---+------+--------------------+-----------+----+---------+---------------+---------------+
| id|degree|          department|     school|  id|     name|graduateProgram|  spark program|
+---+------+--------------------+-----------+----+---------+---------------+---------------+
|  0|Master|School of Informa...|UC Berkeley|   0|Abdullah |              0|          [100]|
|  1|   PhD|                EECS|UC Berkeley|   1|   Harun |              1|[500, 250, 100]|
|  1|   PhD|                EECS|UC Berkeley|   2|    Omer |              1|     [250, 100]|
|  2|Master|                EECS|UC Berkeley|null|     null|           null|           null|
+---+------+--------------------+-----------+----+---------+---------------+---------------+

+----+---------+---------------+---------------+---+------+--------------------+-----------+
|  id|     name|graduateProgram|  spark program| id|degree|          department|     school|
+----+---------+---------------+---------------+---+------+----------

In [23]:
#Left Semi LeftAnti
jointype="left_semi"
graduateProgram.join(person,joinExpression,jointype).show()
jointype="left_anti"
graduateProgram.join(person,joinExpression,jointype).show()



+---+------+--------------------+-----------+
| id|degree|          department|     school|
+---+------+--------------------+-----------+
|  0|Master|School of Informa...|UC Berkeley|
|  1|   PhD|                EECS|UC Berkeley|
+---+------+--------------------+-----------+

+---+------+----------+-----------+
| id|degree|department|     school|
+---+------+----------+-----------+
|  2|Master|      EECS|UC Berkeley|
+---+------+----------+-----------+



In [24]:
#Natural Joins-Cross joins
jointype="cross"
graduateProgram.join(person,joinExpression,jointype).show()

+---+------+--------------------+-----------+---+---------+---------------+---------------+
| id|degree|          department|     school| id|     name|graduateProgram|  spark program|
+---+------+--------------------+-----------+---+---------+---------------+---------------+
|  0|Master|School of Informa...|UC Berkeley|  0|Abdullah |              0|          [100]|
|  1|   PhD|                EECS|UC Berkeley|  1|   Harun |              1|[500, 250, 100]|
|  1|   PhD|                EECS|UC Berkeley|  2|    Omer |              1|     [250, 100]|
+---+------+--------------------+-----------+---+---------+---------------+---------------+



Spark cluster üzerinde çalışırken dikkat etmemiz gereken diğer bir husus ise spark join işlemleri için cluster içerisinde nodelar arasında gerekli veri alışverişini yapıyor ancak eğer çok çok büyük tablolar ile daha küçük ölçekterki tabloları join işlemine tabi tutuyorsak bu durumda spark küçük tabloyu tüm büyük veri setinin parçaları bulunan node'lara birer kopyasını gönderiyor ve join işlemini tamamlıyor.

Bu kısımda kısaca spark üzerinde join nasıl yapılır onu görmüş olduk.

<b>resource<b/>=https://github.com/databricks/spark-csv